Import Libraries


In [27]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import FastText
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

Read Data

In [28]:
df1=pd.read_csv('/content/Art.csv')
df1

,Unnamed: 0,Article Title,Category
0,0,\n\nWorks by Picasso and others stolen in Spai...,Art
1,1,\n\nSalwa and Maged at the Art Corner Gallery ...,Art
2,2,\n\nA tribute to Abdalla Gohar\n\n\n\nOn 1 Dec...,Art
3,3,"\n\nRaw, untamed, and compelling: ‘Flesh’ by S...",Art
4,4,\n\nFrom water to mosaics\n\n\n\nThe Gezira Ce...,Art
...,...,...,...
10452,10452,Netflix’s first Arabic film production remains...,Art
10453,10453,Disney+ to launch in Egypt this summer\n\n\n\n...,Art
10454,10454,China gives 'Fight Club' new ending where auth...,Art
10455,10455,Egypt’s Actors’ Syndicate defends Mona Zaki as...,Art


In [29]:
df2=pd.read_csv('/content/Economy.csv')
df2

,Unnamed: 0,Article Title,Category
0,0,"\n\nChinese demand, Mideast turmoil boost ship...",economy
1,1,\n\nEgypt government to issue LE1 bn in zero c...,economy
2,2,\n\nFirst Egyptian Samsung factory starts prod...,economy
3,3,\n\nUPDATED: Egyptian and Arab stocks soar as ...,economy
4,4,\n\nGreek high school teachers to strike over ...,economy
...,...,...,...
9995,9975,Egypt imposes three-month export ban on cookin...,economy
9996,9976,"Egypt bans export of wheat, other staples for ...",economy
9997,9977,"Egypt’s annual inflation hit 10% in February, ...",economy
9998,9978,INTERVIEW: EBRD supports Egypt’s green transfo...,economy


In [30]:
df3=pd.read_csv('/content/Sports.csv')
df3

,Unnamed: 0,Article Title,Category
0,0,\n\nFans in shock after Leicester City helicop...,sports
1,1,\n\nLeicester City football club owner was on ...,sports
2,2,\n\nLeicester City football club owner's helic...,sports
3,3,\n\nRonaldo scores twice as Juve overcome frig...,sports
4,4,\n\nKlopp hails Liverpool's '20-point win' ove...,sports
...,...,...,...
9975,9955,Germany Bundesliga results & scorers (21st mat...,sports
9976,9956,Barcelona go fourth after winning six-goal thr...,sports
9977,9957,Dortmund humiliated by Leverkusen in Haaland's...,sports
9978,9958,"Back from injury, 1st Liverpool goal for teen ...",sports


Make the 3 datasets in one dataset

In [31]:
# Concatenate the DataFrames along the rows axis
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv('combined.csv', index=False)

In [32]:
combined_df

,Unnamed: 0,Article Title,Category
0,0,\n\nWorks by Picasso and others stolen in Spai...,Art
1,1,\n\nSalwa and Maged at the Art Corner Gallery ...,Art
2,2,\n\nA tribute to Abdalla Gohar\n\n\n\nOn 1 Dec...,Art
3,3,"\n\nRaw, untamed, and compelling: ‘Flesh’ by S...",Art
4,4,\n\nFrom water to mosaics\n\n\n\nThe Gezira Ce...,Art
...,...,...,...
30432,9955,Germany Bundesliga results & scorers (21st mat...,sports
30433,9956,Barcelona go fourth after winning six-goal thr...,sports
30434,9957,Dortmund humiliated by Leverkusen in Haaland's...,sports
30435,9958,"Back from injury, 1st Liverpool goal for teen ...",sports


Replace every \n

In [33]:
combined_df['Article Title'] = combined_df['Article Title'].str.replace(r'\n+', ' ')


<ipython-input-33-dd76cb045183>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_df['Article Title'] = combined_df['Article Title'].str.replace(r'\n+', ' ')


In [34]:
combined_df

,Unnamed: 0,Article Title,Category
0,0,Works by Picasso and others stolen in Spain ...,Art
1,1,Salwa and Maged at the Art Corner Gallery Ex...,Art
2,2,"A tribute to Abdalla Gohar On 1 December, Kha...",Art
3,3,"Raw, untamed, and compelling: ‘Flesh’ by Sami...",Art
4,4,From water to mosaics The Gezira Centre for M...,Art
...,...,...,...
30432,9955,Germany Bundesliga results & scorers (21st mat...,sports
30433,9956,Barcelona go fourth after winning six-goal thr...,sports
30434,9957,Dortmund humiliated by Leverkusen in Haaland's...,sports
30435,9958,"Back from injury, 1st Liverpool goal for teen ...",sports


Drop unnessecery column

In [35]:
combined_df.drop('Unnamed: 0',axis=1, inplace=True)

In [36]:
combined_df

,Article Title,Category
0,Works by Picasso and others stolen in Spain ...,Art
1,Salwa and Maged at the Art Corner Gallery Ex...,Art
2,"A tribute to Abdalla Gohar On 1 December, Kha...",Art
3,"Raw, untamed, and compelling: ‘Flesh’ by Sami...",Art
4,From water to mosaics The Gezira Centre for M...,Art
...,...,...
30432,Germany Bundesliga results & scorers (21st mat...,sports
30433,Barcelona go fourth after winning six-goal thr...,sports
30434,Dortmund humiliated by Leverkusen in Haaland's...,sports
30435,"Back from injury, 1st Liverpool goal for teen ...",sports


Make all characters in a lowercase

In [37]:
combined_df["Article Title"] = combined_df["Article Title"].str.lower()


Call lemmatizer and download punkt,wordner and stopWords

In [38]:
# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

making Limitizer

In [39]:

# Tokenize and lemmatize each word in each title
combined_df['Article Title'] = combined_df['Article Title'].apply(lambda title: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(title)]))

# Print the DataFrame to verify the changes
print(combined_df)


                                           Article Title Category
0      work by picasso and others stolen in spain pic...      Art
1      salwa and maged at the art corner gallery exhi...      Art
2      a tribute to abdalla gohar on 1 december , kha...      Art
3      raw , untamed , and compelling : ‘ flesh ’ by ...      Art
4      from water to mosaic the gezira centre for mod...      Art
...                                                  ...      ...
30432  germany bundesliga result & scorer ( 21st matc...   sports
30433  barcelona go fourth after winning six-goal thr...   sports
30434  dortmund humiliated by leverkusen in haaland '...   sports
30435  back from injury , 1st liverpool goal for teen...   sports
30436  manchester city , chelsea , spur get favourabl...   sports

[30437 rows x 2 columns]


Drop Stop words

In [40]:
# Remove stop words
stop_words = set(stopwords.words("english"))
combined_df["Article Title"] = combined_df["Article Title"].apply(lambda x: " ".join([word for word in nltk.word_tokenize(x) if word.lower() not in stop_words]))

In [41]:
combined_df

,Article Title,Category
0,work picasso others stolen spain picasso art s...,Art
1,salwa maged art corner gallery exhibiting pain...,Art
2,"tribute abdalla gohar 1 december , khan al mag...",Art
3,"raw , untamed , compelling : ‘ flesh ’ samir f...",Art
4,water mosaic gezira centre modern art exhibit ...,Art
...,...,...
30432,germany bundesliga result & scorer ( 21st matc...,sports
30433,barcelona go fourth winning six-goal thriller ...,sports
30434,dortmund humiliated leverkusen haaland 's abse...,sports
30435,"back injury , 1st liverpool goal teen elliott ...",sports


Makibg embedded with many ways

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Assuming combined_df["Article Title"] contains the article titles
embeddings_spacy = []

for title in combined_df["Article Title"]:
    doc = nlp(title)
    embeddings_spacy.append(doc.vector)

In [43]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(combined_df["Article Title"])
embeddings_tfidf = X.toarray()

In [44]:
y = combined_df["Category"]
X = embeddings_tfidf  # Assuming embeddings_word2vec contains the Word2Vec embeddings

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:



nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predictions
y_pred = nb_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy:", accuracy)

In [ ]:
sentences = [title.split() for title in combined_df["Article Title"]]
model_fasttext = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

embeddings_fasttext = [model_fasttext.wv[title.split()].mean(axis=0) for title in combined_df["Article Title"]]


In [ ]:
y = combined_df["Category"]
X = embeddings_fasttext  # Assuming embeddings_word2vec contains the Word2Vec embeddings

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:



nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predictions
y_pred = nb_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy:", accuracy)

In [ ]:
sentences = [title.split() for title in combined_df["Article Title"]]
model_word2vec = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
embeddings_word2vec = [model_word2vec.wv[title.split()].mean(axis=0) for title in combined_df["Article Title"]]

In [ ]:
y = combined_df["Category"]
X = embeddings_word2vec  # Assuming embeddings_word2vec contains the Word2Vec embeddings

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:



nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predictions
y_pred = nb_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

In [ ]:
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf' or 'poly' based on your data
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy:", accuracy)